# Katib 실습
katib을 사용하여 training code `01-covid19-katib-train.py`를 hyper parameter tunning한다

# Method 1. Host에서 command로 실행

### Step 1. docker 명령으로 build (정석)
**Host에 ssh 접속**하여 `01-covid19-katib-train.py`를 업로드한 후

아래와 같이 docker build하고 push한다

```sh
REGISTRY=kubeflow-registry.default.svc.cluster.local:30000
TAG=$REGISTRY/katib-job:latest

cat << EOF | docker build -t $TAG -f - . 
FROM brightfly/kubeflow-jupyter-lab:tf2.0-gpu
COPY 01-covid19-katib-train.py /app/
CMD ['python', '/app/01-covid19-katib-train.py']
EOF

docker push $TAG
```

### Setp 2. kubectl로 apply
Host에 ssh 접속하여 아래와 같이 kubectl apply 실행
```sh
kubectl apply -f 03-covid19-katib-random.yaml
```

# Method 2. JupyterNotebook에서 실행

### Step 1. fairing으로 build
아래 셀을 실행하여 fairing으로 image build하면 image tag명이난수로 생성된다
(기존 fairing에서 build만 실행하기 실습과 동일한 코드이다)

In [ ]:
from kubeflow.fairing.preprocessors.base import BasePreProcessor 
from kubeflow.fairing.builders.append.append import AppendBuilder

preproc = BasePreProcessor(
    input_files = ['01-covid19-katib-train.py'], 
    #path_prefix = '/myapp', # default: /app
    #command = ['python3'],  # default: python
)    

builder = AppendBuilder(
    preprocessor = preproc,
    base_image = 'brightfly/kubeflow-jupyter-lab:tf2.0-gpu',
    image_name = 'katib-job',
    registry = 'kubeflow-registry.default.svc.cluster.local:30000',
    push = True,
)

builder.build()

print(f'Published Docker image with tag: "{builder.image_tag}"')

### Step 2. katib UI에서 submit
katib UI의 submit 화면에서 `03-covid19-katib-random.yaml` 내용을 copy & paste하고 submit 버튼 클릭

Step 1.에서 fairing이 생성한 image tag명으로 `03-covid19-katib-random.yaml`을 수정한다
```yaml
apiVersion: "kubeflow.org/v1alpha3"
kind: Experiment
...
spec:
...
  trialTemplate:
    goTemplate:
...
          spec:
            template:
              spec:
                containers:
                - name: {{.Trial}}
                  image: kubeflow-registry.default.svc.cluster.local:30000/katib-job:latest  # 여기를 수정
```

### 관련 SDK API
- https://kubeflow-fairing.readthedocs.io/en/latest/source/kubeflow.fairing.preprocessors.html#module-kubeflow.fairing.preprocessors.base
- https://kubeflow-fairing.readthedocs.io/en/latest/source/kubeflow.fairing.builders.append.html#module-kubeflow.fairing.builders.append.append

### 참고 Notebook Preprocessor
```
from kubeflow.fairing.preprocessors.converted_notebook import ConvertNotebookPreprocessor
preproc = ConvertNotebookPreprocessor(
    notebook_file="example.ipynb"
)    
```